In [ ]:
import torch
import torch.nn as nn
from conformer import ConformerBlock  # from sooftware's Conformer repo

class ConformerClassifier(nn.Module):
    def __init__(self, input_dim=128, num_classes=58, num_blocks=4, hidden_dim=144, num_heads=4, dropout=0.1):
        super(ConformerClassifier, self).__init__()

        self.input_proj = nn.Conv1d(input_dim, hidden_dim, kernel_size=1)

        self.conformer_blocks = nn.Sequential(*[
            ConformerBlock(
                dim=hidden_dim,
                dim_head=hidden_dim // num_heads,
                heads=num_heads,
                ff_mult=4,
                conv_expansion_factor=2,
                conv_kernel_size=31,
                dropout=dropout
            )
            for _ in range(num_blocks)
        ])

        self.pooling = nn.AdaptiveAvgPool1d(1) 
        self.classifier = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = x.squeeze(1)
        x = self.input_proj(x)
        x = x.transpose(1, 2)
        x = self.conformer_blocks(x)
        x = x.transpose(1, 2)
        x = self.pooling(x).squeeze(-1)
        x = self.classifier(x) 
        return x


ImportError: cannot import name 'ConformerBlock' from 'conformer' (c:\Users\15592\Downloads\lazs-final-project\conformer\__init__.py)

In [14]:
import sys
sys.path.append('./')  # Ensure project root is in the Python path

from conformer.conformer.encoder import ConformerEncoder
from conformer.conformer.encoder import ConformerBlock
import librosa
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Dataset setup
class TimbreDataset(Dataset):
    def __init__(self, root_dir, instrument_to_label, sample_rate=22050, duration_ms=500, n_mels=128):
        self.samples = []
        self.labels = []
        self.sr = sample_rate
        self.clip_len = int(sample_rate * duration_ms / 1000)
        self.n_mels = n_mels
        self.instrument_to_label = instrument_to_label

        for instrument, label in instrument_to_label.items():
            inst_dir = os.path.join(root_dir, instrument)
            for file in os.listdir(inst_dir):
                if file.endswith(".wav"):
                    self.samples.append(os.path.join(inst_dir, file))
                    self.labels.append(label)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        file_path = self.samples[idx]
        label = self.labels[idx]
        y, _ = librosa.load(file_path, sr=self.sr)
        y_trim, _ = librosa.effects.trim(y, top_db=30)
        clip = y_trim[:self.clip_len]
        if len(clip) < self.clip_len:
            clip = np.pad(clip, (0, self.clip_len - len(clip)))
        mel = librosa.feature.melspectrogram(y=clip, sr=self.sr, n_mels=self.n_mels)
        log_mel = librosa.power_to_db(mel, ref=np.max)
        return torch.tensor(log_mel).unsqueeze(0), torch.tensor(label)

# Model definition
from conformer import ConformerBlock

class ConformerClassifier(nn.Module):
    def __init__(self, input_dim=128, num_classes=58, num_blocks=4, hidden_dim=144, num_heads=4, dropout=0.1):
        super(ConformerClassifier, self).__init__()
        self.input_proj = nn.Conv1d(input_dim, hidden_dim, kernel_size=1)
        self.conformer_blocks = nn.Sequential(*[
            ConformerBlock(
                dim=hidden_dim,
                dim_head=hidden_dim // num_heads,
                heads=num_heads,
                ff_mult=4,
                conv_expansion_factor=2,
                conv_kernel_size=31,
                dropout=dropout
            ) for _ in range(num_blocks)
        ])
        self.pooling = nn.AdaptiveAvgPool1d(1)
        self.classifier = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = x.squeeze(1)
        x = self.input_proj(x)
        x = x.transpose(1, 2)
        x = self.conformer_blocks(x)
        x = x.transpose(1, 2)
        x = self.pooling(x).squeeze(-1)
        return self.classifier(x)

# Training and evaluation
classes = ['violin', 'viola', 'cello', 'double-bass', 'clarinet', 'bass-clarinet', 'saxophone', 'flute',
           'oboe', 'bassoon', 'contrabassoon', 'cor-anglais', 'french-horn', 'trombone', 'trumpet', 'tuba',
           'english-horn', 'guitar', 'mandolin', 'banjo', 'chromatic-percussion']

instrument_to_label = {name: idx for idx, name in enumerate(classes)}
dataset = TimbreDataset("data/all-samples", instrument_to_label)
train_size = int(0.7 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_ds, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=0)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ConformerClassifier(input_dim=128, num_classes=len(classes)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(5):  # keep small while testing
    model.train()
    running_loss = 0.0
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device, dtype=torch.float)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()
        preds = model(x_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

# Evaluation
model.eval()
all_preds = []
all_targets = []
with torch.no_grad():
    for x_batch, y_batch in val_loader:
        x_batch = x_batch.to(device, dtype=torch.float)
        y_batch = y_batch.to(device)
        preds = model(x_batch)
        pred_labels = torch.argmax(preds, dim=1)
        all_preds.extend(pred_labels.cpu().numpy())
        all_targets.extend(y_batch.cpu().numpy())

precision = precision_score(all_targets, all_preds, average='macro')
recall = recall_score(all_targets, all_preds, average='macro')
f1 = f1_score(all_targets, all_preds, average='macro')
print(f"\nPrecision: {precision:.3f}, Recall: {recall:.3f}, F1 Score: {f1:.3f}")

# Confusion matrix
cm = confusion_matrix(all_targets, all_preds)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt="d", xticklabels=classes, yticklabels=classes, cmap="magma")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.tight_layout()
plt.show()


ImportError: cannot import name 'ConformerBlock' from 'conformer' (c:\Users\15592\Downloads\lazs-final-project\conformer\__init__.py)